<a href="https://colab.research.google.com/github/Golovin-Andrey/3dmotif_search/blob/main/motif_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install ProDy
!pip install -U ProDy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.2 MB/s 
     |████████████████████████████████| 3.1 MB 43.4 MB/s 
  Created wheel for ProDy: filename=ProDy-2.3.1-cp38-cp38-linux_x86_64.whl size=11750346 sha256=d7cd688debfef0e8c48a44f60a35acc402570b379ed6e45ee514199e9c4cb62c
  Stored in directory: /root/.cache/pip/wheels/fe/70/79/df801d96836c4ab4d3850a6bba7370cc21a66599b5bc05b616
Successfully built ProDy


In [ ]:
#@title Import nessesary stuff

import prody as pd
import numpy as np
from os import listdir
import pickle
import matplotlib.pyplot as plt

#import itertools
import networkx as nx
import itertools
import time
from io import StringIO
import pandas
import random
from copy import copy
import os, sys
import shutil

In [ ]:
#@title Mount Drive

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

try:
  os.makedirs('/content/drive/MyDrive/motif')
except:
  print ('motif folder already exists')


Mounted at /content/drive
motif folder already exists


In [ ]:
cd /content/drive/MyDrive/motif

In [ ]:
#@title Setup workdir

try:
  os.makedirs('/content/drive/MyDrive/motif/scripts')
  os.makedirs('/content/drive/MyDrive/motif/dump')
except:
  pass
t=time.time()
os.makedirs('/content/drive/MyDrive/motif/'+str(t))

print ('executing at /content/drive/MyDrive/motif/'+str(t))

In [ ]:
#@title Download scripts
%%bash

cd /content/drive/MyDrive/motif/scripts/
rm *
wget https://vsb.fbb.msu.ru/share/motif/pdb_to_hash.py 
wget https://vsb.fbb.msu.ru/share/motif/reorg_hash.py 
wget https://vsb.fbb.msu.ru/share/motif/bins.py 
wget https://vsb.fbb.msu.ru/share/motif/search_save.py
wget https://vsb.fbb.msu.ru/share/motif/download_bins.bash

wget https://vsb.fbb.msu.ru/share/motif/m.pdb 
wget https://vsb.fbb.msu.ru/share/motif/4hfz.pdb
wget https://vsb.fbb.msu.ru/share/motif/6ggn.pdb

In [ ]:
#@title Optional: upload custom scaffolds
#@markdown -  upload and process pdb to search motive in 
#@markdown -  alternatively - search in CATH S40 Non-redundant dataset 

from google.colab import files
use_custom_scaffolds = False #@param {type:"boolean"}

use_example = True #@param {type:"boolean"}

if use_example == True:
  uploaded = ['6ggn.pdb','4hfz.pdb']
  for i in uploaded:
    shutil.move('/content/drive/MyDrive/motif/scripts/'+i, '/content/drive/MyDrive/motif/dump/'+i)

if use_example == True:
  for i in uploaded:
    print (i)
    !cd /content/drive/MyDrive/motif/dump ; python /content/drive/MyDrive/motif/scripts/pdb_to_hash.py /content/drive/MyDrive/motif/dump/{i} /content/drive/MyDrive/motif/{t}/hash /content/drive/MyDrive/motif/{t}/dict
    !python /content/drive/MyDrive/motif/scripts/reorg_hash.py /content/drive/MyDrive/motif/{t}/hash/ /content/drive/MyDrive/motif/{t}/hash_reorg/

else:
  if use_custom_scaffolds == True:
  
    uploaded = files.upload()
    for i in uploaded:
      shutil.move('./'+i, '/content/drive/MyDrive/motif/dump/'+i)


  if use_custom_scaffolds == True:
    for i in uploaded:
      print (i)
      !cd /content/drive/MyDrive/motif/dump ; python /content/drive/MyDrive/motif/scripts/pdb_to_hash.py /content/drive/MyDrive/motif/dump/{i} /content/drive/MyDrive/motif/{t}/hash /content/drive/MyDrive/motif/{t}/dict
      !python /content/drive/MyDrive/motif/scripts/reorg_hash.py /content/drive/MyDrive/motif/{t}/hash/ /content/drive/MyDrive/motif/{t}/hash_reorg/



In [ ]:
#@title Upload motif pdb

 

use_example_motif = True #@param {type:"boolean"}

if not use_example_motif:

  motif_list = files.upload()
  for i in motif_list:

    shutil.move('./'+i, '/content/drive/MyDrive/motif/'+str(t)+'/motif.pdb')
else:
  shutil.move('/content/drive/MyDrive/motif/scripts//m.pdb', '/content/drive/MyDrive/motif/'+str(t)+'/motif.pdb')


In [ ]:
#@title Select search tolerance


dx = 1 #@param [1,2,3,4,5] {type:"raw"}
ax = 1 #@param [1,2,3,4,5,6] {type:"raw"}
bx = 1 #@param [1,2,3,4,5,6] {type:"raw"}
tx = 1 #@param [1,2,3] {type:"raw"}
nx = 1 #@param [1,2,3] {type:"raw"}
mx = 1 #@param [1,2,3] {type:"raw"}

!python /content/drive/MyDrive/motif/scripts/bins.py {t} {dx} {ax} {bx} {tx} {nx} {mx}

In [ ]:
#@title Download/copy bins for search
if use_example == True: 
  use_custom_scaffolds == True
!bash /content/drive/MyDrive/motif/scripts/download_bins.bash {use_custom_scaffolds} {t} 2>/dev/null

In [ ]:
#@title Defining found pdb structures
pdblist = []
if use_custom_scaffolds == True:
  for i in uploaded:
    print (i)
    pdblist = pdblist + [str(i)[:-4]]
else:
  ls = os.listdir('/content/drive/MyDrive/motif/'+str(t)+'/pulled_bins/')
  ppp = {}
  for i in ls:
    ppp[i] = os.listdir('/content/drive/MyDrive/motif/'+str(t)+'/pulled_bins/'+i)
  for k in sorted(ppp, key=lambda k: len(ppp[k]), reverse=True):
        pdblist = ppp[k]

  
print (pdblist)

In [ ]:
#@title Saving results

%%bash -s "$t" "$use_custom_scaffolds" "$pdblist" 

if [[ $2 == 'False' ]]; 
then cd /content/drive/MyDrive/motif/dump/; for i in $3; do b=$(echo $i | sed 's/[^0-9a-z]*//g'); echo $b ; wget https://vsb.fbb.msu.ru/share/motif/dompdb/$b ; done;
fi

for i in $3; do b=$(echo $i | sed 's/[^0-9a-z]*//g'); echo $b ; python /content/drive/MyDrive/motif/scripts/search_save.py $b $1 ;done

In [ ]:
print ("results:", '/content/drive/MyDrive/motif/'+str(t)+'/results')

!ls /content/drive/MyDrive/motif/{t}/results